# Causal Simulations: parametric g-formula (g-computation)
Simulated data sets demonstrating the unbiasedness of the implemented g-formula estimator under several different data generating mechanisms. 1000 samples of 2000 individuals are used to demonstrate the g-formula for the average causal effect of a time-fixed exposure on outcomes at a single time point. All parametric models are correctly specified. Briefly described below are the main features of each data generating mechanism.

Data-generating mechanism 1:
- Binary outcome

Data-generating mechanism 2:
- Normally distributed outcome

Data-generating mechanism 3:
- Binary outcome with interaction terms

Data-generating mechanism 4:
- Continuous outcome with informative censoring

Data-generating mechanism 4:
- Binary outcome with missing treatment data
- Demonstrated g-formula with IPMW to account for missingness of treatment data

*Notes*: confidence intervals for g-formula are calculated using a non-parametric bootstrap procedure with 500 samples with replacement.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import logistic

from zepid.causal.gformula import TimeFixedGFormula
from zepid.causal.ipw import IPMW

np.random.seed(20191203)

sample_size = 2000
sim_size = 1000
bstrap_size = 200

In [2]:
def dgm(version, n=10000000):
    """Generates one of five different data generating mechanisms for simulations. 
    Generates a target population 10,000,000 individuals.
    
    Version 1: binary outcome
    Version 2: continuous outcome
    Version 3: binary outcome
    Version 4: continuous outcome with censoring
    Version 5: binary outcome with missing treatment information
    """
    df = pd.DataFrame()
    if version == 1:
        # Creating confounders
        df['W'] = np.random.normal(10, 3, size=n)
        df['L'] = np.random.binomial(n=1, p=0.4, size=n)
        # Treatment model
        df['A'] = np.random.binomial(n=1, p=logistic.cdf(-0.5*df['W'] + 0.02*df['W']*df['W'] + 5*df['L']), size=n)
        # Outcome models
        df['Y1'] = np.random.binomial(n=1, p=logistic.cdf(-3 + 0.2*df['W'] - 3*df['L']), size=n)
        df['Y0'] = np.random.binomial(n=1, p=logistic.cdf(0.2*df['W'] - 3*df['L']), size=n)        
        df['Y'] = np.where(df['A'] == 1, df['Y1'], df['Y0'])
        return df
    
    if version == 2:
        # Creating confounders
        df['Q'] = np.random.normal(size=n)
        df['Z'] = np.random.binomial(n=1, p=0.8, size=n)
        # Treatment model
        df['A'] = np.random.binomial(n=1, p=logistic.cdf(0.75 + 1.5*df['Q'] - 4*df['Z']), size=n)
        # Outcome models
        df['Y1'] = 129 + 0.2*df['Q'] - 0.01*df['Q']*df['Q'] + 5*df['Z'] + np.random.normal(0, 2, size=n)
        df['Y0'] = 122 + 0.2*df['Q'] - 0.01*df['Q']*df['Q'] - 4*df['Z'] + np.random.normal(0, 2, size=n)
        df['Y'] = np.where(df['A'] == 1, df['Y1'], df['Y0'])
        return df

    if version == 3:
        # Creating confounders
        df['X'] = np.random.normal(size=n)
        df['B'] = np.random.binomial(n=1, p=0.6, size=n)
        df['C'] = np.random.binomial(n=1, p=0.3, size=n)
        # Treatment model
        df['A'] = np.random.binomial(n=1, p=logistic.cdf(-1.75 - 1.5*df['X'] + 3*df['B'] + 2*df['C'] 
                                                         - 5*df['B']*df['C']), size=n)
        # Outcome models
        df['Y1'] = np.random.binomial(n=1, p=logistic.cdf(-2 - 0.2*df['X'] + 3*df['B'] + 1.5*df['C'] 
                                                          + 0.1*df['X']*df['C']), size=n)
        df['Y0'] = np.random.binomial(n=1, p=logistic.cdf(-5 - 0.2*df['X'] + 3*df['B'] + 1.5*df['C'] 
                                                          + 0.1*df['X']*df['C']), size=n)
        df['Y'] = np.where(df['A'] == 1, df['Y1'], df['Y0'])
        return df
    
    if version == 4:
        # Creating confounders
        df['R'] = np.random.normal(size=n)
        df['S'] = np.random.normal(size=n)
        df['T'] = np.random.binomial(n=1, p=0.4, size=n)
        # Treatment model
        df['A'] = np.random.binomial(n=1, p=logistic.cdf(-1.195 - 1.5*df['S'] + 2*df['T'] 
                                                         + 0.3*df['S']*df['T']), size=n)
        # Outcome models
        df['Y1'] = 27 + df['R'] + df['S'] - 0.2*df['R']*df['S'] - 3*df['T'] + np.random.normal(size=n)
        df['Y0'] = 27 + df['R'] + df['S'] - 0.2*df['R']*df['S'] - 3*df['T'] + np.random.normal(size=n)
        df['Y'] = np.where(df['A'] == 1, df['Y1'], df['Y0'])
        # Censoring model
        df['My'] = np.random.binomial(n=1, p=logistic.cdf(-1.975 + 0.8*df['A'] + 0.1*df['R']), size=n)
        df['Y'] = np.where(df['My'] == 1, np.nan, df['Y'])
        return df
    
    if version == 5:
        # Creating confounders
        df['G'] = np.random.normal(5, 1, size=n)
        df['H'] = np.random.binomial(n=1, p=0.4, size=n)
        df['K'] = np.random.binomial(n=1, p=0.7, size=n)
        # Treatment model
        df['A'] = np.random.binomial(n=1, p=logistic.cdf(0.5*df['G'] + 3*df['H'] - 5*df['K']), size=n)
        # Outcome models
        df['Y1'] = np.random.binomial(n=1, p=logistic.cdf(-2 + 0.25*df['G'] - 4*df['H']), size=n)
        df['Y0'] = np.random.binomial(n=1, p=logistic.cdf(0.5 + 0.25*df['G'] - 5*df['H']), size=n)        
        df['Y'] = np.where(df['A'] == 1, df['Y1'], df['Y0'])
        # Missing model
        df['Ma'] = np.random.binomial(n=1, p=logistic.cdf(-2.5 + 3*df['K'] - 1*df['H']), size=n)
        df['A'] = np.where(df['Ma'] == 1, np.nan, df['A'])
        return df


## Data-generating mechanism 1

In [3]:
df = dgm(version=1)
truth = np.mean(df['Y1'] - df['Y0'])
bias_naive = []
bias_gcomp = []
ci_gcomp = []

for i in range(sim_size):
    dfs = df.sample(n=sample_size)
    # naive
    bias_naive.append(np.mean(dfs.loc[dfs['A'] == 1, 'Y']) - np.mean(dfs.loc[dfs['A'] == 0, 'Y']) - truth) 

    # g-computation
    gcomp = TimeFixedGFormula(dfs, exposure='A', outcome='Y')
    gcomp.outcome_model(model='A + W + L', print_results=False)
    gcomp.fit(treatment='all')
    r_all = gcomp.marginal_outcome
    gcomp.fit(treatment='none')
    r_none = gcomp.marginal_outcome
    rd_point = r_all - r_none
    bias_gcomp.append(rd_point - truth)
    
    # Bootstrap confidence intervals
    rd_results = []
    for i in range(bstrap_size):
        s = dfs.sample(n=dfs.shape[0],replace=True)
        g = TimeFixedGFormula(s,exposure='A',outcome='Y')
        g.outcome_model(model='A + W + L', print_results=False)
        g.fit(treatment='all')
        r_all = g.marginal_outcome
        g.fit(treatment='none')
        r_none = g.marginal_outcome
        rd_results.append(r_all - r_none)

    se = np.std(rd_results, ddof=1)
    if rd_point - 1.96*se < truth < rd_point + 1.96*se:
        ci_gcomp.append(1)
    else:
        ci_gcomp.append(0)

    
results = pd.DataFrame()
results['bias_naive'] = bias_naive
results['bias_gcomp'] = bias_gcomp
results['ci_gcomp'] = ci_gcomp
results.describe()

,bias_naive,bias_gcomp,ci_gcomp
count,1000.000000,1000.000000,1000.000000
mean,-0.332801,-0.000646,0.940000
std,0.012970,0.037634,0.237606
min,-0.369324,-0.118220,0.000000
25%,-0.341021,-0.026439,1.000000
50%,-0.333420,-0.001521,1.000000
75%,-0.324362,0.024929,1.000000
max,-0.289293,0.111311,1.000000


## Data-generating mechanism 2

In [4]:
df = dgm(version=2)
df['Q_sq'] = df['Q']**2
truth = np.mean(df['Y1'] - df['Y0'])
bias_naive = []
bias_gcomp = []
ci_gcomp = []

for i in range(sim_size):
    dfs = df.sample(n=sample_size)
    # naive
    bias_naive.append(np.mean(dfs.loc[dfs['A'] == 1, 'Y']) - np.mean(dfs.loc[dfs['A'] == 0, 'Y']) - truth) 

    # g-computation
    gcomp = TimeFixedGFormula(dfs, exposure='A', outcome='Y', outcome_type='normal')
    gcomp.outcome_model(model='A + Q + Q_sq + Z + A:Z', print_results=False)
    gcomp.fit(treatment='all')
    r_all = gcomp.marginal_outcome
    gcomp.fit(treatment='none')
    r_none = gcomp.marginal_outcome
    rd_point = r_all - r_none
    bias_gcomp.append(rd_point - truth)

    # Bootstrap confidence intervals
    rd_results = []
    for i in range(bstrap_size):
        s = dfs.sample(n=dfs.shape[0],replace=True)
        g = TimeFixedGFormula(s,exposure='A',outcome='Y', outcome_type='normal')
        g.outcome_model(model='A + Q + Q_sq + Z + A:Z', print_results=False)
        g.fit(treatment='all')
        r_all = g.marginal_outcome
        g.fit(treatment='none')
        r_none = g.marginal_outcome
        rd_results.append(r_all - r_none)

    se = np.std(rd_results, ddof=1)
    if rd_point - 1.96*se < truth < rd_point + 1.96*se:
        ci_gcomp.append(1)
    else:
        ci_gcomp.append(0)

    
results = pd.DataFrame()
results['bias_naive'] = bias_naive
results['bias_gcomp'] = bias_gcomp
results['ci_gcomp'] = ci_gcomp
results.describe()

,bias_naive,bias_gcomp,ci_gcomp
count,1000.000000,1000.000000,1000.000000
mean,-1.730508,-0.004190,0.941000
std,0.168040,0.190024,0.235743
min,-2.253104,-0.636066,0.000000
25%,-1.848701,-0.125307,1.000000
50%,-1.727176,-0.002361,1.000000
75%,-1.617917,0.123748,1.000000
max,-1.208832,0.630788,1.000000


## Data-generating mechanism 3

In [5]:
df = dgm(version=3)
truth = np.mean(df['Y1'] - df['Y0'])
bias_naive = []
bias_gcomp = []
ci_gcomp = []

for i in range(sim_size):
    dfs = df.sample(n=sample_size)
    # naive
    bias_naive.append(np.mean(dfs.loc[dfs['A'] == 1, 'Y']) - np.mean(dfs.loc[dfs['A'] == 0, 'Y']) - truth) 

    # g-computation
    gcomp = TimeFixedGFormula(dfs, exposure='A', outcome='Y')
    gcomp.outcome_model(model='A + X + B + C + X:C', print_results=False)
    gcomp.fit(treatment='all')
    r_all = gcomp.marginal_outcome
    gcomp.fit(treatment='none')
    r_none = gcomp.marginal_outcome
    rd_point = r_all - r_none
    bias_gcomp.append(rd_point - truth)

    # Bootstrap confidence intervals
    rd_results = []
    for i in range(bstrap_size):
        s = dfs.sample(n=dfs.shape[0],replace=True)
        g = TimeFixedGFormula(s,exposure='A',outcome='Y')
        g.outcome_model(model='A + X + B + C + X:C', print_results=False)
        g.fit(treatment='all')
        r_all = g.marginal_outcome
        g.fit(treatment='none')
        r_none = g.marginal_outcome
        rd_results.append(r_all - r_none)

    se = np.std(rd_results, ddof=1)
    if rd_point - 1.96*se < truth < rd_point + 1.96*se:
        ci_gcomp.append(1)
    else:
        ci_gcomp.append(0)

results = pd.DataFrame()
results['bias_naive'] = bias_naive
results['bias_gcomp'] = bias_gcomp
results['ci_gcomp'] = ci_gcomp
results.describe()

,bias_naive,bias_gcomp,ci_gcomp
count,1000.000000,1000.000000,1000.000000
mean,0.075013,0.001281,0.938000
std,0.019072,0.018551,0.241276
min,0.017110,-0.057282,0.000000
25%,0.062054,-0.010423,1.000000
50%,0.075231,0.001735,1.000000
75%,0.088559,0.014152,1.000000
max,0.129561,0.056841,1.000000


## Data-generating mechanism 4

In [6]:
df = dgm(version=4)
truth = np.mean(df['Y1'] - df['Y0'])
bias_naive = []
bias_gcomp = []
ci_gcomp = []

for i in range(sim_size):
    dfs = df.sample(n=sample_size)
    # naive
    bias_naive.append(np.mean(dfs.loc[dfs['A'] == 1, 'Y']) - np.mean(dfs.loc[dfs['A'] == 0, 'Y']) - truth) 

    # g-computation
    gcomp = TimeFixedGFormula(dfs, exposure='A', outcome='Y', outcome_type='normal')
    gcomp.outcome_model(model='A + R + S + T + R:S', print_results=False)
    gcomp.fit(treatment='all')
    r_all = gcomp.marginal_outcome
    gcomp.fit(treatment='none')
    r_none = gcomp.marginal_outcome
    rd_point = r_all - r_none
    bias_gcomp.append(rd_point - truth)

    # Bootstrap confidence intervals
    rd_results = []
    for i in range(bstrap_size):
        s = dfs.sample(n=dfs.shape[0],replace=True)
        g = TimeFixedGFormula(s,exposure='A',outcome='Y', outcome_type='normal')
        g.outcome_model(model='A + R + S + T + R:S', print_results=False)
        g.fit(treatment='all')
        r_all = g.marginal_outcome
        g.fit(treatment='none')
        r_none = g.marginal_outcome
        rd_results.append(r_all - r_none)

    se = np.std(rd_results, ddof=1)
    if rd_point - 1.96*se < truth < rd_point + 1.96*se:
        ci_gcomp.append(1)
    else:
        ci_gcomp.append(0)

results = pd.DataFrame()
results['bias_naive'] = bias_naive
results['bias_gcomp'] = bias_gcomp
results['ci_gcomp'] = ci_gcomp
results.describe()

,bias_naive,bias_gcomp,ci_gcomp
count,1000.000000,1000.000000,1000.00000
mean,-1.950274,-0.001770,0.95400
std,0.102179,0.061856,0.20959
min,-2.270969,-0.191938,0.00000
25%,-2.020985,-0.044965,1.00000
50%,-1.947366,0.001052,1.00000
75%,-1.877705,0.041178,1.00000
max,-1.642286,0.196489,1.00000


## Data-generating mechanism 5

In [7]:
df = dgm(version=5)
truth = np.mean(df['Y1'] - df['Y0'])
bias_naive = []
bias_gcomp1 = []
ci_gcomp1 = []
bias_gcomp2 = []
ci_gcomp2 = []

for i in range(sim_size):
    dfs = df.sample(n=sample_size)
    # naive
    bias_naive.append(np.mean(dfs.loc[dfs['A'] == 1, 'Y']) - np.mean(dfs.loc[dfs['A'] == 0, 'Y']) - truth) 
    
    # g-computation
    gcomp = TimeFixedGFormula(dfs.dropna(), exposure='A', outcome='Y')
    gcomp.outcome_model(model='A + G + H + A:H', print_results=False)
    gcomp.fit(treatment='all')
    r_all = gcomp.marginal_outcome
    gcomp.fit(treatment='none')
    r_none = gcomp.marginal_outcome
    rd_point1 = r_all - r_none
    bias_gcomp1.append(rd_point1 - truth)

    # Bootstrap confidence intervals
    rd_results = []
    for i in range(200):
        s = dfs.sample(n=dfs.shape[0], replace=True)
        g = TimeFixedGFormula(s.dropna(), exposure='A', outcome='Y')
        g.outcome_model(model='A + G + H + A:H', print_results=False)
        g.fit(treatment='all')
        r_all = g.marginal_outcome
        g.fit(treatment='none')
        r_none = g.marginal_outcome
        rd_results.append(r_all - r_none)

    se = np.std(rd_results, ddof=1)
    if rd_point1 - 1.96*se < truth < rd_point1 + 1.96*se:
        ci_gcomp1.append(1)
    else:
        ci_gcomp1.append(0)
    
    # calculating IPMW for A
    ipmw = IPMW(dfs, missing_variable='A')
    ipmw.regression_models('K + H', print_results=False)
    ipmw.fit()
    dfs['ipmw'] = ipmw.Weight

    # g-computation with IPMW
    gcomp = TimeFixedGFormula(dfs.dropna(), exposure='A', outcome='Y', weights='ipmw')
    gcomp.outcome_model(model='A + G + H + A:H', print_results=False)
    gcomp.fit(treatment='all')
    r_all = gcomp.marginal_outcome
    gcomp.fit(treatment='none')
    r_none = gcomp.marginal_outcome
    rd_point2 = r_all - r_none
    bias_gcomp2.append(rd_point2 - truth)

    # Bootstrap confidence intervals
    rd_results = []
    for i in range(bstrap_size):
        s = dfs.sample(n=dfs.shape[0],replace=True)

        ipmw = IPMW(s, missing_variable='A')
        ipmw.regression_models('K + H', print_results=False)
        ipmw.fit()
        s['ipmw'] = ipmw.Weight
        
        g = TimeFixedGFormula(s.dropna(),exposure='A',outcome='Y', weights='ipmw')
        g.outcome_model(model='A + G + H + A:H', print_results=False)
        g.fit(treatment='all')
        r_all = g.marginal_outcome
        g.fit(treatment='none')
        r_none = g.marginal_outcome
        rd_results.append(r_all - r_none)

    se = np.std(rd_results, ddof=1)
    if rd_point2 - 1.96*se < truth < rd_point2 + 1.96*se:
        ci_gcomp2.append(1)
    else:
        ci_gcomp2.append(0)

    
results = pd.DataFrame()
results['bias_naive'] = bias_naive
results['bias_gcomp1'] = bias_gcomp1
results['ci_gcomp1'] = ci_gcomp1
results['bias_gcomp2'] = bias_gcomp2
results['ci_gcomp2'] = ci_gcomp2
results.describe()

,bias_naive,bias_gcomp1,ci_gcomp1,bias_gcomp2,ci_gcomp2
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.132528,0.036163,0.557000,0.000404,0.946000
std,0.026103,0.020492,0.496989,0.021814,0.226131
min,-0.213794,-0.034965,0.000000,-0.069789,0.000000
25%,-0.149608,0.022274,0.000000,-0.013656,1.000000
50%,-0.133124,0.036370,1.000000,-0.000110,1.000000
75%,-0.115525,0.050110,1.000000,0.014911,1.000000
max,-0.040229,0.097152,1.000000,0.065256,1.000000
